# Εξόρυξη δεδομένων

Αλίκη Τσαμοπούλου Δαμιανή sdi1900193 \
Παναγιώτης Μαντάς sdi1400294
---

Η άσκηση έγινε με το VS code και python 3.10.14\
Η πιο συμαντική παραδοχή που πείραμε μετά απο συζήτη στο eclass είναι ότι στο αρχείο listings δεν υπάρχουν πολλαπλότυπα,\
πράγμα που στην πραγματικότητα δεν συμβαίνει καθώς ενώνοντας τα δεδομένα μέσα στο αρχείο μπορεί να υπάρχουν, 
διπλότυπα ή και πολλαπλότυπα.\
Αυτό έχει ως αποτέλεσμα να επηρεάζει την ακρίβεια των αποτελεσμάτων.


## Δήλωση βιβλιοθηκών

In [2]:
import pandas as pd
import numpy as np
import warnings
import csv
import matplotlib.pyplot as plt
from collections import Counter
from scipy.interpolate import interp1d
import folium
from wordcloud import WordCloud, STOPWORDS
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter
import math
from langdetect import detect
from transformers import pipeline


warnings.filterwarnings("ignore", category=pd.errors.DtypeWarning)


2024-06-05 11:40:00.780676: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-05 11:40:00.830340: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-05 11:40:00.830378: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-05 11:40:00.831649: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-05 11:40:00.838845: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-05 11:40:02.057863: W tensorflow/compiler/tf2tensorrt/utils/py_utils.

## Δήλωση Σταθερών

In [27]:
TARGET_TRAINING_DATASET_SIZE = 200

## Προεπεξεργασία

### Προετοιμασία των reviews

#### Φόρτωση στη μνήμη

In [3]:
reviews = {
    "2019": [],
    "2023": [],
}

file_path_2019 = f'2019/april/reviews.csv'
file_path_2023 = f'2023/september/reviews.csv'
    
reviews["2019"] = pd.read_csv(file_path_2019)
reviews["2023"] = pd.read_csv(file_path_2023)

#### Φιλτράρισμα γνωρισμάτων

In [4]:
filter_columns = {
    "2019": [
        'id', 'comments'
    ],
    "2023": [
        'id', 'comments'
    ]
}

working_datasets_reviews = {
    "2019": [],
    "2023": []
}

print("Shapes on load: ")

for year, dataset in reviews.items():
    working_datasets_reviews[year] = reviews[year][filter_columns[year]]
    working_datasets_reviews[year].reset_index(drop=True, inplace=True)

    print(working_datasets_reviews[year].shape)

    working_datasets_reviews[year].to_csv(f'output/train_{year}.csv', quoting=csv.QUOTE_NONNUMERIC, quotechar='"', escapechar='\\', encoding="utf-8", index=False, doublequote=False)

print("Data saved to output")

Shapes on load: 
(287433, 2)
(579991, 2)
Data saved to output


Στα δεδομένα περνάμε και την στήλη neighouhood_cleansed διότι σε αυτή την στήλη είναι ήδη καθαρισμένα τα δεδομένα.\
Φτιάχνουμε και το αρχείο που ζητήθηκε train_2019. \
Φτιάχνουμε και μια στήλη month για να μπορούμε αργότερα να ξέρουμε απο πιο μήνα προήλθαν τα δεδομένα.\

Στο πεδίο price αφαιρέσαμε το σύμβολο του δολλαρίου και αφαιρέσαμε το , ώστε να περάσει ως αριθμός στο panda και να μπορούμε να το χειριστούμε αναλόγος

#### Προσθήκη γνωρίσματος sentiment

In [5]:
for year, dataset in working_datasets_reviews.items():
    dataset.loc[:,'sentiment'] = None


/tmp/ipykernel_31316/3584559531.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.loc[:,'sentiment'] = None
/tmp/ipykernel_31316/3584559531.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.loc[:,'sentiment'] = None


#### Επιλογή πλειάδων που είναι σε αγγλικά

In [8]:
# Function to check if a column name is in English
def is_english(column_name):
    try:
        return detect(column_name) == 'en'
    except:
        return False
    
cleaned_datasets_reviews = {
    "2019": [],
    "2023": []
}

print("Shapes before cleaning empty rows: ")

# ------------------for development ####
for year, dataset in working_datasets_reviews.items():
    working_datasets_reviews[year] = dataset.sample(500, replace=False)    
# ------------------------------

for year, dataset in working_datasets_reviews.items():
    print(year, dataset.shape)

for year, dataset in working_datasets_reviews.items():
    df = dataset.dropna(subset=['comments'])  # Specify columns to check for empty strings

    df_filtered = df[df['comments'].apply(is_english)]
    
    cleaned_datasets_reviews[year] = df_filtered


print("Shapes after cleaning empty rows: ")

for year, dataset in cleaned_datasets_reviews.items():
    print(year, dataset.shape)

    dataset.to_csv(f'output/reviews_{year}_english.csv', quoting=csv.QUOTE_NONNUMERIC, quotechar='"', escapechar='\\', encoding="utf-8", index=False, doublequote=False)


Shapes before cleaning empty rows: 
2019 (500, 3)
2023 (500, 3)
Shapes after cleaning empty rows: 
2019 (405, 3)
2023 (354, 3)


### Προετοιμασία των listings

In [10]:
month_map = {
    "2019": ['april'],
    "2023": ['september'],
}

complete_datasets_listings = {
    "2019": [],
    "2023": []
}


for year, months in month_map.items():
    print(year, months)

    data_frames = []

    for month in months:
        file_path = f'{year}/{month}/listings.csv'
        df = pd.read_csv(file_path)
        df['month'] = month
        data_frames.append(df)
        print("Dimensions for " , file_path, " are " , df.shape)

        df.to_csv(f'output/{year}_{month}_listings.csv', index=False)

    complete_datasets_listings[year] = pd.concat(data_frames, ignore_index=True)
    complete_datasets_listings[year].reset_index(drop=True, inplace=True)

for year, dataset in complete_datasets_listings.items():
    print("Dataset", year, "shape: ", complete_datasets_listings[year].shape)

print("Listings data loaded")



2019 ['april']
Dimensions for  2019/april/listings.csv  are  (9661, 107)
2023 ['september']
Dimensions for  2023/september/listings.csv  are  (12955, 76)
Dataset 2019 shape:  (9661, 107)
Dataset 2023 shape:  (12955, 76)
Data loaded


#### Φιλτράρισμα γνωρισμάτων στο listings

In [15]:
filter_columns = {
    "2019": [
        'id', 'review_scores_rating', 'number_of_reviews'
    ],
    "2023": [
        'id', 'review_scores_rating', 'number_of_reviews'
    ]
}

working_datasets_listings = {
    "2019": [],
    "2023": []
}

for year, dataset in complete_datasets_listings.items():
    working_datasets_listings[year] = complete_datasets_listings[year][filter_columns[year]]
    working_datasets_listings[year].reset_index(drop=True, inplace=True)

    print("Year", year, "Total columns: ", len(filter_columns[year]))
    print(working_datasets_listings[year].shape)

    working_datasets_listings[year].to_csv(f'output/train_{year}.csv', quoting=csv.QUOTE_NONNUMERIC, quotechar='"', escapechar='\\', encoding="utf-8", index=False, doublequote=False)

print("Data saved to output")



Year 2019 Total columns:  3
(9661, 3)
Year 2023 Total columns:  3
(12955, 3)
Data saved to output


#### Ταξινόμηση βάσει review_scores_rating

In [25]:
for year, dataset in working_datasets_listings.items():
    working_datasets_listings[year] = working_datasets_listings[year].dropna()
    working_datasets_listings[year] = working_datasets_listings[year].sort_values(by='review_scores_rating', ascending=True)

working_datasets_listings['2019']



,id,review_scores_rating,number_of_reviews
9002,32706562,20.0,1
8962,32654936,20.0,1
1883,14152595,20.0,1
7740,30192195,20.0,1
3121,19104215,20.0,1
...,...,...,...
3101,19047757,100.0,8
3098,19041864,100.0,7
3082,19002022,100.0,1
3128,19126329,100.0,1


In [36]:
working_datasets_listings_positive = {
    "2019": [],
    "2023": []
}

working_datasets_listings_negative = {
    "2019": [],
    "2023": []
}


for year, dataset in working_datasets_listings.items():
    working_datasets_listings_positive[year] = working_datasets_listings[year].head(TARGET_TRAINING_DATASET_SIZE//2)
    working_datasets_listings_negative[year] = working_datasets_listings[year].tail(TARGET_TRAINING_DATASET_SIZE//2)

    print("Year", year)
    print("Positive shape: ", working_datasets_listings_positive[year].shape)
    print("Negative shape: ", working_datasets_listings_negative[year].shape)


# TODO:

-- find listing ids
-- find reviews of those listing ids
-- predict sentiment for those listing ids
-- create a subset for each prediction
-- merdge subsets
-- save the subset for the next step...


Year 2019
Positive shape:  (100, 3)
Negative shape:  (100, 3)
Year 2023
Positive shape:  (100, 3)
Negative shape:  (100, 3)


### Σΰνθεση του τελικού αρχείου εκπαίδευσης

TODO: Ένωση των 2/3 αρχείων που θα έχουν εγγραφές ανά συναίσθημα

In [11]:
sample_datasets = {
    "2019": [],
    "2023": []
}

for year, dataset in cleaned_datasets_reviews.items():
    sample_datasets[year] = dataset.sample(200, replace=False)


print("Shapes after sampling: ")

for year, dataset in sample_datasets.items():
    print(year, dataset.shape)


Shapes after sampling: 
2019 (200, 3)
2023 (200, 3)


#### Προσθήκη γνωρίσματος cleansed_comments

In [12]:
sample_datasets_cleansed = {
    "2019": [],
    "2023": []
}

for year, dataset in sample_datasets.items():
    dataset.loc[:,'cleansed_comments'] = dataset.loc[:,'comments']
    sample_datasets_cleansed[year] = dataset


### Υπολογισμός συνόλου με bias

#### Προεπεξεργασία πεδίου comments

In [13]:
CLEANR = re.compile('<.*?>') #remove html tags
pattern = r'\b\d+m2\b' # remove apartment size
special = r'[-!@#$%^&*()_+={}\[\]:;"\'|<,>.?/~`]' # remove special character
symbols = re.compile("["
                     u"\U0001F600-\U0001F64F"  # Emoticons
                     u"\U0001F300-\U0001F5FF"  # Symbols & pictographs
                     u"\U0001F680-\U0001F6FF"  # Transport & map symbols
                     u"\U0001F1E0-\U0001F1FF"  # Flags (iOS)
                     u"\U00002500-\U00002BEF"  # Chinese/Japanese/Korean characters
                     u"\U00002702-\U000027B0"
                     u"\U00002702-\U000027B0"
                     u"\U000024C2-\U0001F251"
                     u"\U0001f926-\U0001f937"
                     u"\U00010000-\U0010ffff"
                     u"\u2640-\u2642"
                     u"\u2600-\u2B55"
                     u"\u200d"
                     u"\u23cf"
                     u"\u23e9"
                     u"\u231a"
                     u"\ufe0f"  # dingbats
                     u"\u3030"
                     "]+", re.UNICODE)

def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in STOPWORDS]
    return ' '.join(filtered_words)

def cleanUp(field):
    if field == None:
        return ""
    field = field.lower()
    field = re.sub(CLEANR, ' ', field)
    # HTTP/HTTPS
    # HASHTAGS

    field = re.sub(r'[^\w\s]', ' ', field) # Remove punctuation
    field = re.sub(r'\d+', ' ', field) # Remove digits
    field = re.sub(r'\s+', ' ', field).strip() # Remove extra whitespaces
    field = re.sub(r'\b\w\b', ' ', field) # Remove single letters
    field = re.sub(pattern, ' ', field) #remove appartment sizes
    field = re.sub(r'\b\w{2}\b', ' ', field) #remove 2 letter words
    field = re.sub(symbols, '', field)
    field = re.sub(special, '', field)
    field = remove_stopwords(field)
    return field

for year, dataset in sample_datasets_cleansed.items():    
    for index, row in dataset.iterrows():
        dataset.at[index, 'cleansed_comments'] = cleanUp(row['cleansed_comments'])

        dataset.to_csv(f'output/train_{year}_sample_cleansed.csv', quoting=csv.QUOTE_NONNUMERIC, quotechar='"', escapechar='\\', encoding="utf-8", index=False, doublequote=False)

d1 = sample_datasets_cleansed['2019']
d2 = sample_datasets_cleansed['2023']        

## Ανάλυση συναισθημάτων

In [ ]:
classifier = pipeline(model="lxyuan/distilbert-base-multilingual-cased-sentiments-student",  return_all_scores=True)


All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


In [ ]:
classifier("The book is on the table.")

[[{'label': 'positive', 'score': 0.45254042744636536},
  {'label': 'neutral', 'score': 0.2954728901386261},
  {'label': 'negative', 'score': 0.25198668241500854}]]